In [ ]:
!pip install pandas
!pip install dask

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import os
# Create a local cluster
# Get the number of physical cores
n_cores = os.cpu_count()
print(n_cores)
# Create a local cluster with one worker per core
cluster = LocalCluster(n_workers=int(n_cores/5), threads_per_worker=1, memory_limit='80GB')

# Connect to the cluster
client = Client(cluster)

# Your code here

import os
import csv

Count_GO = True
E_filter = False
root_terms = set(['GO:0008150', 'GO:0003674', 'GO:0005575'])

128


## read-in result of MMseqs search

In [3]:
# os.chdir('./preprocessing')
#(1,2) identifiers for query and target sequences/profiles, (3) sequence identity, 
#(4) alignment length, (5) number of mismatches, (6) number of gap openings, 
#(7-8, 9-10) domain start and end-position in query and in target, (11) E-value, and (12) bit score.
alnRes = dd.read_csv('./data/alnRes.m8',sep='\t',header=None)
alnRes.columns = ['EntryID', 'cluster', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'Evalue', 'bitscore']

alnRes.head(5)

,EntryID,cluster,col3,col4,col5,col6,col7,col8,col9,col10,Evalue,bitscore
0,Q6ENB0,7f9o_Q,0.931,735,50,0,1,730,43,777,0.000000e+00,1354
1,Q6ENB0,7eu3_F,0.931,735,50,0,1,730,43,777,0.000000e+00,1354
2,Q6ENB0,7wg5_F,0.685,735,229,0,1,729,1,735,1.666977e-320,990
3,Q6ENB0,7wff_F,0.685,735,229,0,1,729,1,735,1.666977e-320,990
4,Q6ENB0,6khi_F,0.442,717,355,0,1,717,1,638,3.163000e-173,563


In [3]:
alnRes.compute().describe([.30])

,col3,col4,col5,col6,col7,col8,col9,col10,Evalue,bitscore
count,1.378189e+07,1.378189e+07,1.378189e+07,13781891.0,1.378189e+07,1.378189e+07,1.378189e+07,1.378189e+07,1.378189e+07,1.378189e+07
mean,4.587226e-01,2.839119e+02,1.482073e+02,0.0,1.857246e+02,4.644306e+02,5.242068e+01,3.282708e+02,1.286453e-05,2.293885e+02
std,2.129429e-01,2.499623e+02,1.467580e+02,0.0,5.471455e+02,5.928421e+02,1.743324e+02,3.059957e+02,7.926572e-05,2.929405e+02
min,1.650000e-01,1.300000e+01,0.000000e+00,0.0,1.000000e+00,1.500000e+01,1.000000e+00,1.500000e+01,0.000000e+00,3.600000e+01
30%,3.190000e-01,1.600000e+02,6.800000e+01,0.0,1.700000e+01,2.480000e+02,4.000000e+00,1.820000e+02,6.822000e-65,9.000000e+01
50%,3.830000e-01,2.410000e+02,1.240000e+02,0.0,4.600000e+01,3.560000e+02,1.100000e+01,2.750000e+02,6.366000e-36,1.460000e+02
max,1.000000e+00,5.332000e+03,3.559000e+03,0.0,3.527700e+04,3.537500e+04,4.865000e+03,5.148000e+03,9.999000e-04,1.046800e+04


In [4]:
alnRes_clu = alnRes.cluster.unique().compute().size
print("add clusters:",alnRes_clu)
alnRes_entry = alnRes.EntryID.unique().compute().size
print("Entry:",alnRes_entry)

add clusters: 570450
Entry: 112542


## read-in kaggle train_terms for expanding

In [5]:
train_terms = dd.read_csv('./data/train_terms.tsv',sep='\t')
print('add GO terms',train_terms.term.unique().compute().size)

add GO terms 31466


In [6]:
GOBPO = list(np.load('./data/train_targets_BPO_top21285.npy',allow_pickle=True))
GOCCO = list(np.load('./data/train_targets_CCO_top2957.npy',allow_pickle=True))
GOMFO = list(np.load('./data/train_targets_MFO_top7224.npy',allow_pickle=True))
GOs = GOBPO + GOCCO + GOMFO
len(GOs)

31466

In [7]:
train_terms = train_terms[train_terms['term'].isin(GOs)]

## Apply filters

In [8]:
alnRes = alnRes[alnRes['Evalue']<1e-6]

In [4]:
if Count_GO:
    try:
        clu_go_count = dd.read_csv('./data/clu_go_count_all.csv')
        clu_go_count = clu_go_count.compute()
    except:
        merged_clu = dd.merge(alnRes, train_terms, on='EntryID',how='left')
        clu_go_count = merged_clu.groupby(['cluster', 'term']).size().reset_index().rename(columns={0: 'freq'})
        clu_go_count = clu_go_count.compute()
        clu_go_count.to_csv('clu_go_count_all.csv')

In [7]:
if Count_GO:
    clu_annot = clu_go_count.cluster.unique().size
    print('add clusters:',clu_annot)
    term_annot = clu_go_count.term.unique().size
    print('add terms:', term_annot)

add clusters: 560066
add terms: 30453


In [5]:
if Count_GO:
    print('del clusters:',clu_go_count[clu_go_count.freq<=1].cluster.unique().size)
    print('del clusters:',clu_go_count[clu_go_count.freq<=2].cluster.unique().size)
    print('add terms:',clu_go_count[clu_go_count.freq>2].term.unique().size)
    print('add terms:',clu_go_count[clu_go_count.freq>5].term.unique().size)
    
    print(clu_go_count[(clu_go_count.freq>1)&(~clu_go_count.term.isin(root_terms))].describe())
    print(clu_go_count[(clu_go_count.freq>2)&(~clu_go_count.term.isin(root_terms))].describe())

del clusters: 549338
del clusters: 557660
add terms: 16376
add terms: 9676
         Unnamed: 0          freq
count  6.718676e+07  6.718676e+07
mean   5.013073e+07  6.755435e+00
std    3.372147e+07  1.221350e+01
min    6.000000e+00  2.000000e+00
25%    2.159522e+07  2.000000e+00
50%    4.526052e+07  3.000000e+00
75%    7.485525e+07  7.000000e+00
max    1.396529e+08  7.680000e+02
         Unnamed: 0          freq
count  4.398507e+07  4.398507e+07
mean   4.036014e+07  9.263881e+00
std    2.884000e+07  1.447874e+01
min    6.000000e+00  3.000000e+00
25%    1.672544e+07  3.000000e+00
50%    3.488898e+07  5.000000e+00
75%    5.965619e+07  9.000000e+00
max    1.396448e+08  7.680000e+02


In [ ]:
if E_filter:    
    # Group the data by the id column
    result = alnRes[alnRes['Evalue']<1e-6]
    grouped = result.groupby('cluster')

    # Define a function to get the two rows with the smallest values in column 10 for each group
    def get_smallest_two(group):
        return group.nsmallest(5, 'Evalue')
    print('filter')
    # Apply the function to each group

    result = grouped.apply(get_smallest_two, meta=result)

    # Reset the index
    result = result.compute()

    result = result.reset_index(drop=True)
    # Display the result
    # print(result)
    result.describe([.30])

In [7]:
if E_filter:
    result.to_csv('./data/alnRes_5byCLU.csv')

In [ ]:
if E_filter:
    filter_clu = result.col1.unique().size
    print("add clusters:",filter_clu)
    print('del clusters:',alnRes_clu-filter_clu)
    filter_entry = result.EntryID.unique().size
    print("Entry:",filter_entry)
    print('del Entry:',alnRes_entry-filter_entry)

add clusters: 560066
del clusters: 10384
Entry: 70669
del Entry: 41873


In [15]:
if E_filter:  
    result2 = alnRes[alnRes['Evalue']<=1e-60]
    result2.compute().describe([.30])

2023-06-18 04:46:52,408 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 804.34 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 04:46:52,491 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 806.05 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 04:46:52,573 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 869.71 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 04:46:52,574 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memo

,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
count,4.429860e+06,4.429860e+06,4.429860e+06,4429860.0,4.429860e+06,4.429860e+06,4.429860e+06,4.429860e+06,4.429860e+06,4.429860e+06
mean,6.287138e-01,4.303339e+02,1.762776e+02,0.0,1.196980e+02,5.446187e+02,3.089267e+01,4.528418e+02,6.994112e-63,4.803307e+02
std,2.296400e-01,3.531053e+02,2.143038e+02,0.0,3.997497e+02,5.335888e+02,1.406221e+02,3.862059e+02,5.869817e-62,4.109411e+02
min,1.910000e-01,8.400000e+01,0.000000e+00,0.0,1.000000e+00,8.400000e+01,1.000000e+00,8.400000e+01,0.000000e+00,2.050000e+02
30%,4.550000e-01,2.780000e+02,5.900000e+01,0.0,4.000000e+00,3.290000e+02,2.000000e+00,2.880000e+02,1.752000e-153,2.890000e+02
50%,5.800000e-01,3.510000e+02,1.340000e+02,0.0,2.200000e+01,4.280000e+02,5.000000e+00,3.630000e+02,6.183000e-109,3.650000e+02
max,1.000000e+00,5.332000e+03,3.559000e+03,0.0,3.314700e+04,3.356700e+04,4.714000e+03,5.148000e+03,1.000000e-60,1.046800e+04


In [6]:
# result = pd.read_csv('./data/alnRes_2byCLU.csv',sep='\t')
# result.describe()

In [7]:
if E_filter:  
    fluster_clu = result2.col1.unique().compute().size
    print("fluster_clus:",fluster_clu)
    fluster_entry = result2.EntryID.unique().compute().size
    print("fluster_Entry:",fluster_entry)

fluster_clus: 405560
fluster_Entry: 70827


In [8]:
if E_filter:  
    print('del clusters:',alnRes_clu-fluster_clu)
    print('del Entry:',alnRes_entry-fluster_entry)

del clusters: 164890
del Entry: 41715


In [7]:
if E_filter:  
    train_chain_go = pd.merge(result[['EntryID','cluster']], train_terms[['EntryID', 'term']],how='left')
if Count_GO:
    train_chain_go = clu_go_count[clu_go_count.freq>2][['cluster', 'term']]

In [8]:
old_data = pd.read_csv('./data/pdb_chain_go.tsv',sep='\t',skiprows=1)
old_data.head(5)

,PDB,CHAIN,SP_PRIMARY,WITH_STRING,EVIDENCE,GO_ID
0,101m,A,P02185,UniProtKB-KW:KW-0479,IEA,GO:0046872
1,101m,A,P02185,UniProtKB-KW:KW-0561,IEA,GO:0005344
2,101m,A,P02185,UniProtKB-KW:KW-0561,IEA,GO:0015671
3,102m,A,P02185,UniProtKB-KW:KW-0479,IEA,GO:0046872
4,102m,A,P02185,UniProtKB-KW:KW-0561,IEA,GO:0005344


In [12]:
_ = old_data.PDB + old_data.CHAIN
print('original clus:',_.unique().size)

original clus: 418330


In [13]:
print('add clus from fluster_clu:' ,len(set(result.col1.unique().compute()).difference(_.unique())))
print('more clus from alnRes:' ,len(set(alnRes.col1.unique().compute()).difference(_.unique())))

AttributeError: 'DataFrame' object has no attribute 'unique'

In [15]:
new_data = pd.DataFrame({
    'PDB': train_chain_go['cluster'].str.split('_').str[0],
    'CHAIN': train_chain_go['cluster'].str.split('_').str[1],
    'SP_PRIMARY': 'Virtul' if Count_GO else train_chain_go['EntryID'],
    'WITH_STRING': 'Kaggle',
    'EVIDENCE': 'train',
    'GO_ID': train_chain_go['term']
})

# Write the new DataFrame to a TSV file
new_data.head(5)

,PDB,CHAIN,SP_PRIMARY,WITH_STRING,EVIDENCE,GO_ID
0,11gs,B,V9HWE9,Kaggle,train,GO:0005829
1,11gs,B,V9HWE9,Kaggle,train,GO:0005622
2,11gs,B,V9HWE9,Kaggle,train,GO:0043229
3,11gs,B,V9HWE9,Kaggle,train,GO:0043226
4,11gs,B,V9HWE9,Kaggle,train,GO:0110165


In [17]:
pdb_chain_go_new = pd.concat([old_data,new_data], axis=0)
pdb_chain_go_new.to_csv('./data/pdb_chain_go_new_5byclu.tsv',index=False,sep='\t')